# Experiment 099: Conservative Blending Toward Training Mean

**Goal**: Reduce the CV-LB intercept by blending predictions toward training mean.

**Rationale**:
- CV-LB relationship: LB = 4.31 × CV + 0.0525 (R² = 0.95)
- Intercept (0.0525) > Target (0.0347) - target is mathematically unreachable
- If test solvents are fundamentally different, extreme predictions hurt LB
- Blending toward training mean may reduce the intercept

**Approach**:
1. Use the best model (GP+MLP+LGBM ensemble from exp_030)
2. Blend predictions toward training mean with different blend factors
3. Test blend factors: 0.1, 0.2, 0.3, 0.4, 0.5
4. If CV is similar but approach is fundamentally different, submit to check intercept

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, output_dim)
        )
    
    def forward(self, x):
        return self.net(x)

print('MLP defined')

MLP defined


In [7]:
# Conservative Blending GP+MLP+LGBM Ensemble
class ConservativeGPMLPLGBMEnsemble:
    """GP + MLP + LGBM ensemble with conservative blending toward training mean."""
    
    def __init__(self, data='single', blend_factor=0.3):
        self.data = data
        self.blend_factor = blend_factor  # How much to blend toward training mean
        self.mixed = (data == 'full')
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = None
        self.mlp = None
        self.lgbm_models = None
        self.train_mean = None  # Store training mean for blending
        
    def train_model(self, X, Y):
        # Store training mean for conservative blending
        self.train_mean = Y.values.mean(axis=0)
        
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        Y_np = Y.values
        
        # Train GP (one per output)
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        self.gp_models = []
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42)
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp = MLP(X_full_scaled.shape[1]).to(device).double()
        optimizer = torch.optim.Adam(self.mlp.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        X_tensor = torch.tensor(X_full_scaled).to(device)
        Y_tensor = torch.tensor(Y_np).to(device)
        
        self.mlp.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp(X_tensor)
            loss = criterion(pred, Y_tensor)
            loss.backward()
            optimizer.step()
        self.mlp.eval()
        
        # Train LGBM (one per output)
        self.lgbm_models = []
        for i in range(3):
            lgbm = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, max_depth=5, 
                                     num_leaves=31, random_state=42, verbose=-1)
            lgbm.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(lgbm)
    
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gp_models])
        
        # MLP predictions
        with torch.no_grad():
            mlp_preds = self.mlp(torch.tensor(X_full_scaled).to(device)).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        
        # Ensemble: weighted average
        ensemble_pred = 0.3 * gp_preds + 0.4 * mlp_preds + 0.3 * lgbm_preds
        
        # CONSERVATIVE BLENDING: Blend toward training mean
        # This is the key change to reduce the CV-LB intercept
        blended_pred = (1 - self.blend_factor) * ensemble_pred + self.blend_factor * self.train_mean
        
        # Clip to valid range [0, 1]
        blended_pred = np.clip(blended_pred, 0, 1)
        
        return torch.tensor(blended_pred)

print('ConservativeGPMLPLGBMEnsemble defined')
print(f'Blend factor: 0.3 (will test multiple values)')

ConservativeGPMLPLGBMEnsemble defined
Blend factor: 0.3 (will test multiple values)


In [8]:
# Test different blend factors
def compute_cv_for_blend_factor(blend_factor, verbose=True):
    """Compute CV score for a given blend factor."""
    
    # Single solvent CV
    X_single, Y_single = load_data("single_solvent")
    split_generator = generate_leave_one_out_splits(X_single, Y_single)
    
    single_mse_list = []
    for fold_idx, split in enumerate(split_generator):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = ConservativeGPMLPLGBMEnsemble(data='single', blend_factor=blend_factor)
        model.train_model(train_X, train_Y)
        
        predictions = model.predict(test_X)
        predictions_np = predictions.detach().cpu().numpy()
        
        mse = np.mean((predictions_np - test_Y.values) ** 2)
        single_mse_list.append(mse)
    
    single_cv = np.mean(single_mse_list)
    
    # Full data CV
    X_full, Y_full = load_data("full")
    split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
    
    full_mse_list = []
    for fold_idx, split in enumerate(split_generator):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = ConservativeGPMLPLGBMEnsemble(data='full', blend_factor=blend_factor)
        model.train_model(train_X, train_Y)
        
        predictions = model.predict(test_X)
        predictions_np = predictions.detach().cpu().numpy()
        
        mse = np.mean((predictions_np - test_Y.values) ** 2)
        full_mse_list.append(mse)
    
    full_cv = np.mean(full_mse_list)
    combined_cv = (single_cv + full_cv) / 2
    
    if verbose:
        print(f"Blend factor {blend_factor}: Single CV = {single_cv:.6f}, Full CV = {full_cv:.6f}, Combined CV = {combined_cv:.6f}")
    
    return single_cv, full_cv, combined_cv

print('CV function defined')

CV function defined


In [9]:
# Test blend factor 0.0 (no blending - baseline)
print("Testing blend factor 0.0 (no blending - baseline)...")
single_cv_0, full_cv_0, combined_cv_0 = compute_cv_for_blend_factor(0.0)
print(f"\nBaseline (no blending): Combined CV = {combined_cv_0:.6f}")

Testing blend factor 0.0 (no blending - baseline)...


Blend factor 0.0: Single CV = 0.010039, Full CV = 0.010128, Combined CV = 0.010084

Baseline (no blending): Combined CV = 0.010084


In [10]:
# Test blend factor 0.1
print("\nTesting blend factor 0.1...")
single_cv_01, full_cv_01, combined_cv_01 = compute_cv_for_blend_factor(0.1)
print(f"Blend factor 0.1: Combined CV = {combined_cv_01:.6f}")


Testing blend factor 0.1...


Blend factor 0.1: Single CV = 0.010831, Full CV = 0.010999, Combined CV = 0.010915
Blend factor 0.1: Combined CV = 0.010915


In [ ]:
# Test blend factor 0.2
print("\nTesting blend factor 0.2...")
single_cv_02, full_cv_02, combined_cv_02 = compute_cv_for_blend_factor(0.2)
print(f"Blend factor 0.2: Combined CV = {combined_cv_02:.6f}")

In [ ]:
# Test blend factor 0.3
print("\nTesting blend factor 0.3...")
single_cv_03, full_cv_03, combined_cv_03 = compute_cv_for_blend_factor(0.3)
print(f"Blend factor 0.3: Combined CV = {combined_cv_03:.6f}")

In [ ]:
# Summary of results
print("\n" + "="*60)
print("SUMMARY: Conservative Blending Results")
print("="*60)
print(f"Blend factor 0.0 (baseline): CV = {combined_cv_0:.6f}")
print(f"Blend factor 0.1: CV = {combined_cv_01:.6f} ({(combined_cv_01 - combined_cv_0) / combined_cv_0 * 100:+.2f}%)")
print(f"Blend factor 0.2: CV = {combined_cv_02:.6f} ({(combined_cv_02 - combined_cv_0) / combined_cv_0 * 100:+.2f}%)")
print(f"Blend factor 0.3: CV = {combined_cv_03:.6f} ({(combined_cv_03 - combined_cv_0) / combined_cv_0 * 100:+.2f}%)")
print("="*60)

# Find best blend factor
results = [
    (0.0, combined_cv_0),
    (0.1, combined_cv_01),
    (0.2, combined_cv_02),
    (0.3, combined_cv_03),
]
best_blend, best_cv = min(results, key=lambda x: x[1])
print(f"\nBest blend factor: {best_blend} with CV = {best_cv:.6f}")
print(f"Baseline CV (exp_030): 0.008298")
print(f"Target LB: 0.0347")

In [ ]:
# Save results
import json

results_dict = {
    'blend_factor_0.0': float(combined_cv_0),
    'blend_factor_0.1': float(combined_cv_01),
    'blend_factor_0.2': float(combined_cv_02),
    'blend_factor_0.3': float(combined_cv_03),
    'best_blend_factor': float(best_blend),
    'best_cv': float(best_cv),
    'baseline_cv': 0.008298,
    'model': 'ConservativeGPMLPLGBMEnsemble'
}

with open('/home/code/experiments/099_conservative_blending/metrics.json', 'w') as f:
    json.dump(results_dict, f, indent=2)

print("Results saved to metrics.json")

## Generate Submission with Best Blend Factor

The following cells follow the official template structure.

**CRITICAL**: The model class in submission cells MUST match the CV computation class (`ConservativeGPMLPLGBMEnsemble`).

In [ ]:
# Set the best blend factor for submission
BEST_BLEND_FACTOR = best_blend
print(f"Using blend factor {BEST_BLEND_FACTOR} for submission")

# Check if we should generate submission
if best_cv < 0.0081:
    print(f"CV {best_cv:.6f} is BETTER than baseline 0.0081!")
    print("Generating submission...")
    GENERATE_SUBMISSION = True
else:
    print(f"CV {best_cv:.6f} is WORSE than baseline 0.0081")
    print("Generating submission anyway to check if intercept changes...")
    GENERATE_SUBMISSION = True  # Generate anyway to check intercept

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ConservativeGPMLPLGBMEnsemble(data='single', blend_factor=BEST_BLEND_FACTOR)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ConservativeGPMLPLGBMEnsemble(data='full', blend_factor=BEST_BLEND_FACTOR)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

# Also save to standard location
import shutil
import os
os.makedirs('/home/submission', exist_ok=True)
shutil.copy("submission.csv", "/home/submission/submission.csv")
print("Submission saved!")
print(f"Shape: {submission.shape}")
print(submission.head())

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################